This Notebook is to compare hyperinflation between different countries to decide which one the group should present.

In [18]:
from pathlib import Path

ROOT = Path.cwd()
while not (ROOT / "README.md").exists() and ROOT != ROOT.parent:
    ROOT = ROOT.parent

if not (ROOT / "README.md").exists():
    raise FileNotFoundError("Repo root not found. Open the project folder in VS Code (WSL) and rerun.")

DATA_DIR = ROOT / "data"
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"

print("ROOT:", ROOT)
print("RAW_DIR exists:", RAW_DIR.exists(), RAW_DIR)
print("PROCESSED_DIR exists:", PROCESSED_DIR.exists(), PROCESSED_DIR)

ROOT: /home/mehdi/BEE2041/bee2041-hyperinflation-project
RAW_DIR exists: True /home/mehdi/BEE2041/bee2041-hyperinflation-project/data/raw
PROCESSED_DIR exists: True /home/mehdi/BEE2041/bee2041-hyperinflation-project/data/processed


In [19]:
import pandas as pd

META_COUNTRY = ROOT /"data"/"raw"/"Metadata_Country_API_FP.CPI.TOTL.ZG_DS2_en_csv_v2_32.csv"
meta = pd.read_csv(META_COUNTRY)
print(meta.columns.tolist())
print(meta.head(3))

['Country Code', 'Region', 'IncomeGroup', 'SpecialNotes', 'TableName', 'Unnamed: 5']
  Country Code                      Region  IncomeGroup  \
0          ABW   Latin America & Caribbean  High income   
1          AFE                         NaN          NaN   
2          AFG  Middle East & North Africa   Low income   

                                        SpecialNotes  \
0                                                NaN   
1  26 countries, stretching from the Red Sea in t...   
2  The reporting period for national accounts dat...   

                     TableName  Unnamed: 5  
0                        Aruba         NaN  
1  Africa Eastern and Southern         NaN  
2                  Afghanistan         NaN  


In [ ]:
import pandas as pd

TIDY = ROOT /"data"/"processed"/"inflation_worldbank_tidy.csv"
META_COUNTRY = ROOT /"data"/"raw"/"Metadata_Country_API_FP.CPI.TOTL.ZG_DS2_en_csv_v2_32.csv"
OUT = ROOT /"data"/"processed"/"country_extremes.csv"

df = pd.read_csv(TIDY)
meta = pd.read_csv(META_COUNTRY)

# In the metadata file, aggregates typically have blank "Region"
# Keep rows where Region is not null => actual countries
meta_countries = meta[meta["Region"].notna()][["Country Code", "TableName", "Region", "IncomeGroup"]].copy()

# Merge to keep only actual countries
df2 = df.merge(meta_countries, left_on="Country Code", right_on="Country Code", how="inner")

summary = (
    df2.groupby(["TableName", "Country Code"], as_index=False)
       .agg(
            max_inflation=("inflation_annual_%", "max"),
            year_of_max=("year", lambda s: int(s.loc[df2.loc[s.index, "inflation_annual_%"].idxmax()])),
            years_over_100=("inflation_annual_%", lambda s: int((s > 100).sum())),
            years_over_500=("inflation_annual_%", lambda s: int((s > 500).sum())),
            years_over_1000=("inflation_annual_%", lambda s: int((s > 1000).sum())),
        )
       .sort_values("max_inflation", ascending=False)
)

summary.to_csv(OUT, index=False)
print("Top 15 countries by peak annual inflation:")
print(summary.head(15).to_string(index=False))
print("\nSaved:", OUT)

Top 15 countries by peak annual inflation:
       TableName Country Code  max_inflation  year_of_max  years_over_100  years_over_500  years_over_1000
Congo, Dem. Rep.          COD   23773.131774         1994              12               6                4
         Bolivia          BOL   11749.639632         1985               5               2                2
            Peru          PER    7481.663611         1990               7               3                2
         Ukraine          UKR    4734.914347         1993               3               2                1
          Angola          AGO    4145.105982         1996              11               4                3
         Armenia          ARM    3373.759443         1994               2               1                1
          Brazil          BRA    2947.732772         1990              14               6                4
         Belarus          BLR    2221.016570         1994               5               3            